# Document-Level Sentiment Analysis using<br>PyTorch and the Intel® Transfer Learning Tool API

This notebook uses the Intel® Transfer Learning Tool to fine-tune a HuggingFace pretrained BERT model for text classification. While this notebook runs on a single node, this workload can also be run in a multinode setting using the TLT CLI. Consult the project documentation and examples to run it using PyTorch distributed training.

## 1. Import dependencies and setup parameters

This notebook assumes that you have already followed the instructions to setup a Pytorch environment with all the dependencies required to run the notebook.

In [ ]:
import intel_extension_for_pytorch as ipex
import numpy as np
import os
import pandas as pd

# tlt imports
from tlt.datasets import dataset_factory
from tlt.models import model_factory
from tlt.utils.file_utils import download_and_extract_zip_file

# Specify a directory for the dataset to be downloaded
dataset_dir = os.environ["DATASET_DIR"] if "DATASET_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "dataset")
     
# Specify a directory for output
output_dir = os.environ["OUTPUT_DIR"] if "OUTPUT_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "output")

print("Dataset directory:", dataset_dir)
print("Output directory:", output_dir)

## 2. Get the model

In this step, we call the Intel Transfer Learning Tool model factory to list supported Huggingface text classification models. This is a list of pretrained models from Huggingface that we tested with our API. Optionally, the `verbose=True` argument can be added to the `print_supported_models()` function call to get more information about each model (such as the links to Huggingface, the original dataset, etc).

In [ ]:
# See a list of available text classification models
model_factory.print_supported_models(use_case='text_classification', framework='pytorch')

Use the TLT model factory to get one of the models listed in the previous cell. The `get_model` function returns a  model object that will later be used for training. For this example, we will use bert-large-uncased.

In [ ]:
model_name = "bert-large-uncased"
framework = "pytorch"

model = model_factory.get_model(model_name, framework, num_classes=2)

print("Model name:", model.model_name)
print("Framework:", model.framework)
print("Use case:", model.use_case)

## 3. Get the dataset

### Option A: Use the Hugging Face catalog

Here we are using the dataset in the [Hugging Face datasets catalog](https://huggingface.co/datasets).

In [ ]:
dataset_name = "sst2"
dataset = dataset_factory.get_dataset(dataset_dir, model.use_case, model.framework, dataset_name,
                                      dataset_catalog="huggingface", shuffle_files=True, 
                                      split=['train', 'validation'])

print(dataset.info)
print("\nClass names:", str(dataset.class_names))

Skip to the next step [4. Preprocess the dataset](#4.-Preprocess-the-dataset) to continue using your own dataset.

### Option B: Download the SST2 dataset
Option B explicitly downloads the `SST-2.zip` file and extracts a `.tsv` file of training data that is tab separated. The dataset factory expects custom text classification input files to have at least two columns where one is the label and the second column is the text/sentence to classify.

For example, the header and first three rows of the file should look similar to this:
```
sentence	label
hide new secretions from the parental units 	0
contains no wit , only labored gags 	0
that loves its characters and communicates something rather beautiful about human nature 	1
```

When using your own dataset, update the path to your dataset directory, as well the other variables with properties about the dataset like the .csv (or .tsv) file name, class names, delimiter, header, and the map function (if string labels need to be translated into numerical values).

In [ ]:
# Modify the variables below to use a different dataset or a csv file on your local system.
dataset_url = "https://dl.fbaipublicfiles.com/glue/data/SST-2.zip"
csv_name = "train.tsv"
delimiter = "\t"
dataset_subdir = os.path.join(dataset_dir, 'SST-2')
# If we don't already have the csv file, download and extract the zip file to get it.
if not os.path.exists(os.path.join(dataset_subdir, csv_name)):
    download_and_extract_zip_file(dataset_url, dataset_dir)

In [ ]:
dataset = dataset_factory.load_dataset(dataset_dir=dataset_subdir, 
                                       use_case="text_classification",
                                       framework="pytorch", csv_file_name=csv_name,
                                       column_names=["sentence", "label"], 
                                       delimiter=delimiter, header=True, label_col=1)

print(dataset.info)
print("\nClass names:", str(dataset.class_names))

In [ ]:
# Create splits for training and validation
dataset.shuffle_split(train_pct=0.75, val_pct=0.25)

## 4. Preprocess the dataset

Once you have your dataset from Option A or Option B above, use the following cell to preprocess the dataset. The dataset subsets are tokenized and then batched.

In [ ]:
dataset.preprocess(model_name, batch_size=32, max_length=55)

## 5. Fine tuning

The TLT model's train function is called with the dataset that was just prepared, along with an output directory for checkpoints, and the number of training epochs.

With the do_eval paramter set to True by default, this step will also show how the model can be evaluated. The model's evaluate function returns a list of metrics calculated from the dataset's validation subset.

### Arguments

#### Required
-  **dataset** (ImageClassificationDataset, required): Dataset to use when training the model
-  **output_dir** (str): Path to a writeable directory for checkpoint files
-  **epochs** (int): Number of epochs to train the model (default: 1)

#### Optional
-  **initial_checkpoints** (str): Path to checkpoint weights to load. If the path provided is a directory, the latest checkpoint will be used.
-  **ipex_optimize** (bool): Optimize the model using Intel® Extension for PyTorch (default: True)

Note: refer to release documentation for an up-to-date list of train arguments and their current descriptions

In [ ]:
history = model.train(dataset, output_dir, epochs=1, ipex_optimize=True)

## 6. Predict

The model's predict function can be called with a sentence.

In [ ]:
result = model.predict("Terrible movie")

print("Predicted score:", float(result))
print("Predicted label:", dataset.get_str_label(float(result)))

## 7. Export the saved model

Lastly, we can call the model export function to generate a saved_model.pb. Each time the model is exported, a new numbered directory is created, which allows serving to pick up the latest model.

In [ ]:
saved_model_dir = model.export(output_dir)

## Citation

```
@inproceedings{socher-etal-2013-recursive,
    title = "Recursive Deep Models for Semantic Compositionality Over a Sentiment Treebank",
    author = "Socher, Richard  and
      Perelygin, Alex  and
      Wu, Jean  and
      Chuang, Jason  and
      Manning, Christopher D.  and
      Ng, Andrew  and
      Potts, Christopher",
    booktitle = "Proceedings of the 2013 Conference on Empirical Methods in Natural Language Processing",
    month = oct,
    year = "2013",
    address = "Seattle, Washington, USA",
    publisher = "Association for Computational Linguistics",
    url = "https://www.aclweb.org/anthology/D13-1170",
    pages = "1631--1642",
}
```